### 0. Install and import packages

`pip install -U crewai crewai[tools] pandas numpy matplotlib python-dotenv requests --no-cache-dir`

In [1]:
# Install dependencies (run once per environment)
import crewai  # noqa: F401
import pandas  # noqa: F401
import numpy  # noqa: F401
import matplotlib  # noqa: F401
import requests  # noqa: F401


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\leowe\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\leowe\anaconda3\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\leowe\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start
    self.io_loop.start()
  File "C:\Users\leowe\anaconda3\Lib\site-pack

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.




A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\leowe\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\leowe\anaconda3\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\leowe\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start
    self.io_loop.start()
  File "C:\Users\leowe\anaconda3\Lib\site-pack

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.




A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\leowe\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\leowe\anaconda3\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\leowe\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start
    self.io_loop.start()
  File "C:\Users\leowe\anaconda3\Lib\site-pack

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\leowe\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\leowe\anaconda3\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\leowe\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start
    self.io_loop.start()
  File "C:\Users\leowe\anaconda3\Lib\site-pack

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



### 1. Basic setup and helpers

In [2]:
# Setup & environment
import os, json, math, warnings, time
from datetime import datetime, timedelta
from typing import Dict, Any
warnings.filterwarnings('ignore')

from dotenv import load_dotenv
load_dotenv()
OPENAI_MODEL_NAME = os.getenv('OPENAI_MODEL_NAME', 'gpt-4o-mini')
ALPHAVANTAGE_API_KEY = os.getenv('ALPHAVANTAGE_API_KEY')
if not ALPHAVANTAGE_API_KEY:
    print('⚠️  Set ALPHAVANTAGE_API_KEY in your environment for data fetching.')
print('Using model:', OPENAI_MODEL_NAME)

Using model: gpt-4o-mini


### 2. Alpha Vantage fetchers, pure-pandas indicators, risk, rule-based signal, plotting

In [3]:
# Tools
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
from crewai.tools import tool

def _ensure_df(df: pd.DataFrame) -> pd.DataFrame:
    if not isinstance(df, pd.DataFrame) or df.empty:
        raise ValueError('Price DataFrame is empty. Check ticker/period/interval.')
    for needed in ['Open','High','Low','Close','Adj Close','Volume']:
        if needed not in df.columns:
            raise ValueError(f'Missing column: {needed}')
    return df

# --- Indicator helpers (pure pandas/NumPy) ---
def ema(series: pd.Series, span: int) -> pd.Series:
    return series.ewm(span=span, adjust=False).mean()

def rsi(series: pd.Series, period: int = 14) -> pd.Series:
    delta = series.diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.ewm(alpha=1/period, min_periods=period, adjust=False).mean()
    avg_loss = loss.ewm(alpha=1/period, min_periods=period, adjust=False).mean()
    rs = avg_gain / (avg_loss + 1e-12)
    out = 100 - (100 / (1 + rs))
    return out

def macd(series: pd.Series, fast: int = 12, slow: int = 26, signal: int = 9):
    fast_ema = ema(series, fast)
    slow_ema = ema(series, slow)
    macd_line = fast_ema - slow_ema
    signal_line = ema(macd_line, signal)
    return macd_line, signal_line

def bollinger(series: pd.Series, period: int = 20, std_mult: float = 2.0):
    mid = series.rolling(window=period).mean()
    sd = series.rolling(window=period).std()
    upper = mid + std_mult * sd
    lower = mid - std_mult * sd
    return lower, mid, upper

def _parse_period_to_rows(period: str, interval: str) -> int:
    # Rough mapping to slice recent rows if API returns larger history
    # Daily rows per period
    approx_days = {
        '1mo': 22, '3mo': 66, '6mo': 126, '1y': 252, '2y': 504, '5y': 1260
    }.get(period, 126)
    if interval.endswith('wk'):
        return max(approx_days // 5, 50)
    if interval.endswith('mo'):
        return max(approx_days // 21, 24)
    if interval.endswith('min'):
        return min(2000, approx_days * (390 // max(1, int(interval.replace('min','')))))
    return approx_days  # daily

def _av_get(url: str, params: dict) -> dict:
    params = dict(params)
    params['apikey'] = ALPHAVANTAGE_API_KEY
    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    data = r.json()
    if 'Error Message' in data or 'Note' in data:
        raise RuntimeError(f"Alpha Vantage error/rate limit: {data}")
    return data

def _alpha_to_df(data: dict, key: str, adjust_close_field: str = '5. adjusted close') -> pd.DataFrame:
    ts = data.get(key, {})
    if not ts:
        raise ValueError('No time series data in response')
    df = pd.DataFrame(ts).T
    df.index = pd.to_datetime(df.index)
    df = df.rename(columns={
        '1. open':'Open','2. high':'High','3. low':'Low',
        '4. close':'Close', adjust_close_field:'Adj Close','6. volume':'Volume'
    })
    for c in ['Open','High','Low','Close','Adj Close','Volume']:
        df[c] = pd.to_numeric(df[c], errors='coerce')
    df = df.sort_index()
    return df[['Open','High','Low','Close','Adj Close','Volume']]

@tool("Fetch OHLCV price history")
def fetch_ohlcv(ticker: str, period: str = '6mo', interval: str = '1d') -> str:
    """Fetch OHLCV using Alpha Vantage. Supports daily/weekly/monthly and basic intraday.
    Returns JSON {csv_path, rows, start, end, last_close}."""
    if not ALPHAVANTAGE_API_KEY:
        raise RuntimeError('ALPHAVANTAGE_API_KEY not set')

    base = 'https://www.alphavantage.co/query'

    # --- Normalize/validate interval -----------------------------------------
    raw_interval = (interval or '').strip()
    il = raw_interval.lower()
    # Canonicalize to one of: 1d, 1wk, 1mo, Xmin
    if il in {'1d', 'd'}:
        canon_interval = '1d'
    elif il in {'1wk', 'wk', 'w'}:
        canon_interval = '1wk'
    elif il in {'1mo', 'mo', 'm'}:
        canon_interval = '1mo'
    elif il.endswith('min'):
        # keep Xmin if X in {1,5,15,30,60}, else default to 60min
        x = il.replace('min', '')
        canon_interval = f"{x if x in {'1','5','15','30','60'} else '60'}min"
    else:
        # default to daily
        canon_interval = '1d'

    # How many rows to keep based on (period, interval)
    rows_target = _parse_period_to_rows(period, canon_interval)

    def _fallback_daily_if_needed():
        # Try DAILY_ADJUSTED first, then DAILY if adjusted fails/rate-limits
        try:
            data = _av_get(base, {
                'function': 'TIME_SERIES_DAILY_ADJUSTED',
                'symbol': ticker,
                'outputsize': 'full'
            })
            df_ = _alpha_to_df(data, 'Time Series (Daily)')
            return df_
        except Exception as e_adj:
            # Fallback to non-adjusted daily
            data2 = _av_get(base, {
                'function': 'TIME_SERIES_DAILY',
                'symbol': ticker,
                'outputsize': 'full'
            })
            ts = data2.get('Time Series (Daily)', {})
            if not ts:
                raise RuntimeError(f"Alpha Vantage DAILY returned no data for {ticker}; prior error: {e_adj}")
            df_ = pd.DataFrame(ts).T
            df_.index = pd.to_datetime(df_.index)
            df_ = df_.rename(columns={
                '1. open':'Open','2. high':'High','3. low':'Low','4. close':'Close','5. volume':'Volume'
            })
            for c in ['Open','High','Low','Close','Volume']:
                df_[c] = pd.to_numeric(df_[c], errors='coerce')
            df_['Adj Close'] = df_['Close']
            df_ = df_.sort_index()[['Open','High','Low','Close','Adj Close','Volume']]
            return df_

    # --- Fetch branch by interval --------------------------------------------
    if canon_interval == '1d':
        df = _fallback_daily_if_needed()

    elif canon_interval == '1wk':
        data = _av_get(base, {
            'function': 'TIME_SERIES_WEEKLY_ADJUSTED',
            'symbol': ticker
        })
        df = _alpha_to_df(data, 'Weekly Adjusted Time Series', adjust_close_field='5. adjusted close')

    elif canon_interval == '1mo':
        data = _av_get(base, {
            'function': 'TIME_SERIES_MONTHLY_ADJUSTED',
            'symbol': ticker
        })
        df = _alpha_to_df(data, 'Monthly Adjusted Time Series', adjust_close_field='5. adjusted close')

    else:
        # intraday Xmin
        intr = canon_interval.replace('min', '')
        data = _av_get(base, {
            'function': 'TIME_SERIES_INTRADAY',
            'symbol': ticker,
            'interval': f'{intr}min',
            'outputsize': 'full',
            'adjusted': 'true'
        })
        ts_key = f'Time Series ({intr}min)'
        ts = data.get(ts_key, {})
        if not ts:
            # Some tenants return different casing or empty on rate-limit
            available_keys = ', '.join(data.keys())
            raise RuntimeError(f"No intraday data for {ticker} at {intr}min. Keys: {available_keys}")
        df = pd.DataFrame(ts).T
        df.index = pd.to_datetime(df.index)
        df = df.rename(columns={
            '1. open':'Open','2. high':'High','3. low':'Low','4. close':'Close','5. volume':'Volume'
        })
        for c in ['Open','High','Low','Close','Volume']:
            df[c] = pd.to_numeric(df[c], errors='coerce')
        df['Adj Close'] = df['Close']
        df = df.sort_index()[['Open','High','Low','Close','Adj Close','Volume']]

    # --- Clean/validate -------------------------------------------------------
    # Drop rows with missing Close
    df = df.dropna(subset=['Close'])
    if df.empty:
        raise RuntimeError(f"Fetched empty dataframe for {ticker} ({canon_interval}, {period}). "
                           f"Likely rate-limit or symbol mismatch.")

    # Slice last N rows per requested period
    if rows_target and len(df) > rows_target:
        df = df.iloc[-rows_target:]

    # --- Save -----------------------------------------------------------------
    # Sanitize ticker for filename (e.g., BRK.B -> BRK-B)
    safe_ticker = str(ticker).replace('/', '-').replace('\\', '-').replace(' ', '').replace(':', '-').replace('.', '-')
    path = f"{safe_ticker}_{period}_{canon_interval}.csv"
    df.to_csv(path)

    info = {
        'csv_path': path,
        'rows': int(df.shape[0]),
        'start': str(pd.to_datetime(df.index[0]).date()),
        'end': str(pd.to_datetime(df.index[-1]).date()),
        'last_close': float(df['Close'].iloc[-1])
    }
    return json.dumps(info)


@tool("Fetch recent news")
def fetch_news(ticker: str, limit: int = 10) -> str:
    """Fetch recent news using Alpha Vantage NEWS_SENTIMENT endpoint (subject to plan limits).
    Returns JSON list of {title, publisher, link, time}."""
    if not ALPHAVANTAGE_API_KEY:
        return json.dumps([])
    url = 'https://www.alphavantage.co/query'
    params = {
        'function':'NEWS_SENTIMENT',
        'tickers': ticker,
        'sort':'LATEST',
        'limit': min(50, max(1, limit)),
        'apikey': ALPHAVANTAGE_API_KEY
    }
    r = requests.get(url, params=params, timeout=30)
    try:
        r.raise_for_status()
        data = r.json()
    except Exception:
        return json.dumps([])
    feed = data.get('feed', [])
    items = []
    for n in feed[:limit]:
        items.append({
            'title': n.get('title'),
            'publisher': (n.get('source') or n.get('authors') or ['']) if isinstance(n.get('authors'), list) else n.get('source'),
            'link': n.get('url'),
            'time': n.get('time_published')
        })
    return json.dumps(items)


def _read_prices(csv_path: str) -> pd.DataFrame:
    """Robustly read OHLCV CSV where the date may be an unnamed index column or a 'Date' column."""
    df = pd.read_csv(csv_path)
    # Choose the datetime column: prefer 'Date', otherwise use the first column
    date_col = 'Date' if 'Date' in df.columns else df.columns[0]
    df[date_col] = pd.to_datetime(df[date_col], errors='coerce', utc=False)
    df = df.dropna(subset=[date_col]).set_index(date_col).sort_index()
    return _ensure_df(df)

def _compute_indicators_core(csv_path: str) -> dict:
    """Pure-Python core that computes indicators and returns a dict (no tool wrapper)."""
    df = _read_prices(csv_path)

    # Indicators
    df['EMA20'] = ema(df['Close'], 20)
    df['EMA50'] = ema(df['Close'], 50)
    macd_line, sig_line = macd(df['Close'])
    df['MACD'] = macd_line
    df['MACD_SIG'] = sig_line
    df['BB_L'], df['BB_M'], df['BB_U'] = bollinger(df['Close'], 20, 2)
    df['RSI'] = rsi(df['Close'], 14)

    if len(df) < 2:
        raise ValueError("Not enough rows to compute crossovers (need ≥ 2 rows).")

    latest = df.iloc[-1]
    prev = df.iloc[-2]
    events = []
    if prev['EMA20'] < prev['EMA50'] and latest['EMA20'] > latest['EMA50']:
        events.append('Bullish EMA20/50 golden cross today')
    if prev['EMA20'] > prev['EMA50'] and latest['EMA20'] < latest['EMA50']:
        events.append('Bearish EMA20/50 death cross today')
    if prev['MACD'] < prev['MACD_SIG'] and latest['MACD'] > latest['MACD_SIG']:
        events.append('Bullish MACD cross above signal')
    if prev['MACD'] > prev['MACD_SIG'] and latest['MACD'] < prev['MACD_SIG']:
        events.append('Bearish MACD cross below signal')
    if latest['Close'] < latest['BB_L']:
        events.append('Price closed below lower Bollinger band (oversold)')
    if latest['Close'] > latest['BB_U']:
        events.append('Price closed above upper Bollinger band (overbought)')

    return {
        'close': float(latest['Close']),
        'ema20': float(latest['EMA20']),
        'ema50': float(latest['EMA50']),
        'macd': float(latest['MACD']),
        'macd_signal': float(latest['MACD_SIG']),
        'rsi': float(latest['RSI']),
        'bb_lower': float(latest['BB_L']),
        'bb_middle': float(latest['BB_M']),
        'bb_upper': float(latest['BB_U']),
        'events': events,
        'csv_path': csv_path
    }

@tool("Compute technical indicators")
def compute_indicators(csv_path: str) -> str:
    """Compute EMA20/EMA50, MACD, RSI, and Bollinger Bands from OHLCV CSV data.
    Detects crossovers and band breakouts, returning latest values and events as JSON."""
    out = _compute_indicators_core(csv_path)
    return json.dumps(out)


@tool("Compute risk metrics")
def compute_risk(csv_path: str) -> str:
    """Compute annualized volatility, max drawdown, and 1-day 95% VaR from OHLCV CSV data.
    Returns JSON with risk metrics and number of observations."""
    df = _read_prices(csv_path)
    rets = df['Close'].pct_change().dropna()
    vol_ann = float(rets.std() * math.sqrt(252))
    cum = (1 + rets).cumprod()
    roll_max = cum.cummax()
    dd = (cum / roll_max) - 1.0
    max_dd = float(dd.min())
    var95 = float(np.percentile(rets, 5))
    out = {
        'vol_annualized': vol_ann,
        'max_drawdown': max_dd,
        'hist_VaR_1d_95': var95,
        'n_days': int(len(rets))
    }
    return json.dumps(out)


@tool("Rule-based technical signal")
def rule_based_signal(csv_path: str) -> str:
    """Generate a BUY/SELL/HOLD signal using simple rules on EMA, MACD, RSI, and Bollinger Bands.
    Returns JSON with signal, score, reasons, and indicator snapshot."""
    ind = _compute_indicators_core(csv_path)  # <-- call the core, not the tool
    score = 0.0
    reasons = []
    if ind['ema20'] > ind['ema50']:
        score += 1; reasons.append('EMA20 > EMA50 (uptrend)')
    else:
        score -= 1; reasons.append('EMA20 < EMA50 (downtrend)')
    if ind['macd'] > ind['macd_signal']:
        score += 1; reasons.append('MACD > signal (bullish momentum)')
    else:
        score -= 1; reasons.append('MACD < signal (bearish momentum)')
    if ind['rsi'] < 30:
        score += 0.5; reasons.append('RSI < 30 (oversold)')
    elif ind['rsi'] > 70:
        score -= 0.5; reasons.append('RSI > 70 (overbought)')
    if ind['close'] < ind['bb_lower']:
        score += 0.5; reasons.append('Below lower band (mean-reversion up)')
    if ind['close'] > ind['bb_upper']:
        score -= 0.5; reasons.append('Above upper band (mean-reversion down)')
    signal = 'HOLD'
    if score >= 1.5:
        signal = 'BUY'
    elif score <= -1.5:
        signal = 'SELL'
    return json.dumps({'signal': signal, 'score': score, 'reasons': reasons, 'indicators': ind})


@tool("Plot price & indicators")
def plot_price_and_indicators(csv_path: str) -> str:
    """Plot closing price with EMA20/50 and Bollinger Bands, saving to PNG.
    Returns the file path of the generated chart."""
    df = _read_prices(csv_path)
    ind = _compute_indicators_core(csv_path)  # ensures indicators exist logically
    df['EMA20'] = ema(df['Close'], 20)
    df['EMA50'] = ema(df['Close'], 50)
    df['BB_L'], df['BB_M'], df['BB_U'] = bollinger(df['Close'], 20, 2)

    plt.figure(figsize=(12,6))
    df['Close'].plot(label='Close')
    df['EMA20'].plot(label='EMA20')
    df['EMA50'].plot(label='EMA50')
    df['BB_U'].plot(label='BB Upper')
    df['BB_L'].plot(label='BB Lower')
    plt.title('Price with EMAs & Bollinger Bands')
    plt.legend()
    out_path = csv_path.replace('.csv', '_chart.png')
    plt.tight_layout(); plt.savefig(out_path); plt.close()
    return out_path



### 3. Define Agents and Tasks

In [4]:
from crewai import Agent, Task, Crew, Process

market_data_analyst = Agent(
    role="Market Data Analyst",
    goal=("Gather and validate OHLCV & headlines for {ticker}. Summarize trend and anomalies."),
    backstory=("Meticulous about data quality, you verify timeframes and note gaps/splits."),
    tools=[fetch_ohlcv, fetch_news],
    allow_delegation=False,
    verbose=True,
)

technical_strategist = Agent(
    role="Technical Strategist",
    goal=("Transform price data into signals using EMA/RSI/MACD/Bollinger and explain rationale."),
    backstory=("Disciplined technician balancing momentum and mean-reversion; you state both sides."),
    tools=[compute_indicators, rule_based_signal, plot_price_and_indicators],
    allow_delegation=False,
    verbose=True,
)

risk_manager = Agent(
    role="Risk Manager",
    goal=("Quantify risk (vol, drawdown, VaR) and propose a conservative risk plan."),
    backstory=("Capital preservation first; you recommend sensible stops, targets, and sizing."),
    tools=[compute_risk],
    allow_delegation=False,
    verbose=True,
)

portfolio_manager = Agent(
    role="Portfolio Manager",
    goal=("Integrate data/signals/risk and decide: BUY/SELL/HOLD for {ticker} now, with confidence."),
    backstory=("Accountable decision-maker who weighs conflicting evidence and avoids bravado."),
    tools=[],
    allow_delegation=True,
    verbose=True,
)

task_collect = Task(
    description=(
        "For ticker {ticker} with period {period} and interval {interval}:\n"
        "1) Use *Fetch OHLCV price history* and capture CSV path.\n"
        "2) Use *Fetch recent news* (top 5).\n"
        "3) Summarize trend (up/down/sideways) and any data anomalies.\n\n"
        "**Return** JSON: {csv_path, rows, date_start, date_end, last_close, headlines[]}"
    ),
    expected_output=(
        "JSON object with csv_path, rows, date_start, date_end, last_close, headlines[]"
    ),
    agent=market_data_analyst,
)

task_tech = Task(
    description=(
        "Using the CSV, call *Compute technical indicators* and *Rule-based technical signal*. Optionally *Plot*.\n\n"
        "**Return** JSON: {close, ema20, ema50, macd, macd_signal, rsi, bb_lower, bb_middle, bb_upper,"
        " events[], rule_signal{signal, score, reasons[]}, chart_path?}"
    ),
    expected_output=(
        "JSON with indicators, notable events, a rule-based signal, and optional chart path"
    ),
    context=[task_collect],
    agent=technical_strategist,
)

task_risk = Task(
    description=(
        "Using the same CSV, call *Compute risk metrics*. Propose stop_loss %, take_profit %, position_size %"
        " based on volatility and drawdown (conservative).\n\n"
        "**Return** JSON: {vol_annualized, max_drawdown, hist_VaR_1d_95, n_days, suggested:{stop_loss_pct,"
        " take_profit_pct, position_size_pct}}"
    ),
    expected_output=(
        "JSON with risk metrics and a conservative risk plan"
    ),
    context=[task_collect],
    agent=risk_manager,

)

task_decide = Task(
    description=(
        "Synthesize all prior outputs into a single recommendation for {ticker}. Choose one: BUY / SELL / HOLD now."
        " Provide 3–5 reasons (include indicators & risk), confidence (0–1), and restate the risk plan.\n\n"
        "**Return** FINAL JSON: {action, confidence, price, reasons[], key_signals[], risk{stop_loss_pct,"
        " take_profit_pct, position_size_pct}}"
    ),
    expected_output=(
        "A clean JSON object with action, confidence, reasons, key_signals, and risk plan"
    ),
    context=[task_collect, task_tech, task_risk],
    agent=portfolio_manager,

)

crew = Crew(
    agents=[market_data_analyst, technical_strategist, risk_manager, portfolio_manager],
    tasks=[task_collect, task_tech, task_risk, task_decide],
    process=Process.sequential,  # <-- avoids the 'Delegate work to coworker' tool entirely
    verbose=True,
)


In [5]:
# Run the crew for a given ticker
TICKER = 'MSTR'      # change me
PERIOD = '1mo'       # e.g., '1y', '2y'
INTERVAL = '1h'      # e.g., '1h', '1wk', '60min'

result = crew.kickoff(inputs={
    'ticker': TICKER,
    'period': PERIOD,
    'interval': INTERVAL,
})
print('\n===== FINAL RECOMMENDATION =====\n')
print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 4a269c62-3024-4cc3-8ce9-de4e69b399c8                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Market Data Analyst                                                                                     │
│                                                                                                                 │
│  Task: For ticker MSTR with period 1mo and interval 1h:                                                         │
│  1) Use *Fetch OHLCV price history* and capture CSV path.                                                       │
│  2) Use *Fetch recent news* (top 5).                                                                            │
│  3) Summarize trend (up/down/sideways) and any data anomalies.                                                  │
│                                                                                                                 │
│  **Return** JSON: {csv_path, rows, date_start, date_end, last_close, headlines[]}                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Market Data Analyst                                                                                     │
│                                                                                                                 │
│  Thought: Thought: I need to first gather the OHLCV price history for MSTR with the specified parameters to     │
│  capture the necessary data for the analysis.                                                                   │
│                                                                                                                 │
│  Using Tool: Fetch OHLCV price history                                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"ticker\": \"MSTR\", \"period\": \"1mo\", \"interval\": \"1h\"}"                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Portfolio Manager                                                                                       │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  {"action":"SELL","confidence":0.8,"reasons":["EMA20 is below EMA50 indicating a downtrend.","MACD is below     │
│  the signal line, confirming bearish momentum.","RSI indicates the stock is oversold but is still in a bearish  │
│  trend, which might reverse. The current price is significantly lower than historical                           │
│  averages."],"key_signals":["EMA20: 355.31","EMA50: 370.96","MACD: -13.54","RSI:                                │
│  25.06"],"risk":{"stop_loss_pct":6.5,"take_profit_pct":12,"position_size_pct":1}}                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 1c0daaf8-5ad4-48c8-bf19-d5ad4d3d12c9                                                                     │
│  Agent: Portfolio Manager                                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 4a269c62-3024-4cc3-8ce9-de4e69b399c8                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: {"action":"SELL","confidence":0.8,"reasons":["EMA20 is below EMA50 indicating a                  │
│  downtrend.","MACD is below the signal line, confirming bearish momentum.","RSI indicates the stock is          │
│  oversold but is still in a bearish trend, which might reverse. The current price is significantly lower than   │
│  historical averages."],"key_signals":["EMA20: 355.31","EMA50: 370.96","MACD: -13.54","RSI:                     │
│  25.06"],"risk":{"stop_loss_pct":6.5,"take_profit_pct":12,"position_size_pct":1}}                               │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


===== FINAL RECOMMENDATION =====

{"action":"SELL","confidence":0.8,"reasons":["EMA20 is below EMA50 indicating a downtrend.","MACD is below the signal line, confirming bearish momentum.","RSI indicates the stock is oversold but is still in a bearish trend, which might reverse. The current price is significantly lower than historical averages."],"key_signals":["EMA20: 355.31","EMA50: 370.96","MACD: -13.54","RSI: 25.06"],"risk":{"stop_loss_pct":6.5,"take_profit_pct":12,"position_size_pct":1}}
